In [1]:
import numpy as np
import nrrd
from build import morse_complex as mc
from utils.functions import *
from skel2graph import create_skeleton_graph, save_skeleton_graph
from util import patchify_voxel, unpatchify_voxel
import pyvista
from skimage.measure import marching_cubes

# autoreload from .py files
%load_ext autoreload
%autoreload 2

## Load Image

In [2]:
img, header = nrrd.read('data/001.nrrd')

## Params

In [3]:
threshold = 0
epsilon = 0
delta = -1

## Do patch operation

In [4]:
patch_size = (48, 48, 48)
pad = (4, 4, 4)
dist_img = distance_transform(1-img).astype(np.float32)
patch_list, start_ind, current_shape, orig_shape = patchify_voxel(dist_img, patch_size, pad)

## Run skeleton on patches

In [5]:
skel_patch_list = []
for i, patch_ in enumerate(patch_list):
    # pad each side by 16 pixel with ones
    print('processing patch', i)
    d = patch_# distance_transform(1-patch_).astype(np.float32)
    MC = mc.MorseComplex(d)
    MC.process_lower_stars(2, 2, 2, 0)

    #################### Cancel pairs ####################
    # MC.cancel_pairs_below(threshold=threshold, print=True)

    #################### Extract Morse Skeleton ####################
    # MC.extract_morse_skeleton_below(threshold=threshold, dimension=3)
    MC.extract_morse_skeleton_parallel_below(threshold=threshold, dimension=3) #TODO: fix bug, hanging randomly
    # MC.extract_morse_skeleton_double_parallel_below(threshold=threshold, dimension=3, batches=64) #TODO: fix bug, hanging randomly

    #################### Extract Morse Graph ####################
    pixels_below = np.array(MC.get_morse_skeleton_below())
    
    dmt_skeleton = np.zeros_like(d)
    if len(pixels_below.shape) == 2:
        dmt_skeleton[pixels_below[:,0], pixels_below[:,1], pixels_below[:,2]] = 1

    # crop the patch to the original size
    dmt_skeleton = dmt_skeleton[pad[0]:-pad[0], pad[1]:-pad[1], pad[2]:-pad[2]]
    skel_patch_list.append(dmt_skeleton)

processing patch 0
Processing lower stars without perturbation...
processing patch 1
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lower stars without perturbation...
processing patch 2
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lower stars without perturbation...
processing patch 3
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lower stars without perturbation...
processing patch 4
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lower stars without perturbation...
processing patch 5
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lower stars without perturbation...
processing patch 6
Computing lower stars without perturbation...
Extracting Morse Skeleton in dimension 3 below 0
Processing lowe

## Reconstruct the whole skeleton from patch

In [6]:
dmt_skeleton = unpatchify_voxel(skel_patch_list, start_ind, patch_size, current_shape, orig_shape)

## Save skeleton as graph

In [7]:
dmt_skeleton_graph = create_skeleton_graph(dmt_skeleton)
save_skeleton_graph(dmt_skeleton_graph, 'images/DMTkeleton_skull.vtp')

## Save segmentation as mesh

In [8]:
mesh_verts, faces, norms, vals = marching_cubes(img>0.0, level=0)
mesh_edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)

mesh_edges = np.concatenate((np.int32(2 * np.ones((mesh_edges.shape[0], 1))), mesh_edges), 1)
mesh = pyvista.UnstructuredGrid(mesh_edges.flatten(), np.array([4] * len(mesh_edges)), mesh_verts)
mesh_structured = mesh.extract_surface().clean()
mesh_structured.save('images/skull_seg.vtp')